# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


##### Воспользуемся TFIDF. Но для начала представим тексты в виде лишь слов, без заглавных букв и знаков препинания

In [4]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df['text'] = df['text'].str.replace('[^a-zA-Z]', ' ')

In [6]:
df['text'] = df['text'].str.lower()

In [7]:
df

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m s...,0
2,hey man i m really not trying to edit war it...,0
3,more i can t make any real suggestions on im...,0
4,you sir are my hero any chance you remember...,0
...,...,...
159566,and for the second time of asking when ...,0
159567,you should be ashamed of yourself that is a ...,0
159568,spitzer umm theres no actual article for pr...,0
159569,and it looks like it was actually you who put ...,0


##### Разделим фрейм на выборки и обучим счетчик TFIDF

In [8]:
features = df['text']
target = df['toxic']

In [9]:
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size = 0.25, random_state = 123)

In [10]:
vect = TfidfVectorizer(stop_words = stopwords, 
                       lowercase = True, 
                       min_df = 0.001)

In [11]:
tridf = vect.fit_transform(train_features)

##### Используем пару моделей и GridSearch

In [12]:
model = LR()
model.fit(tridf, train_target)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
test_idf = vect.transform(test_features)

In [14]:
x = model.predict(test_idf)

In [15]:
f1_score(test_target, x)

0.7344530186109851

##### Метрика пока ниже необходимого, воспользуемся GridSearch

In [16]:
parametrs = {
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter' : range(100, 201, 50)
            }

In [17]:
grid = GridSearchCV(LR(), parametrs, cv = 5)
grid.fit(tridf, train_target)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_iter': range(100, 201, 50),
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [18]:
grid.best_params_

{'max_iter': 100, 'solver': 'saga'}

##### Тут я столкнулся с некоторой несостыковкой гиперпараметров, но заметил что тем не менее можно добиться улучшения метрики

In [19]:
parametrs = {
    'penalty' : ['l1', 'l2'],
    'max_iter' : range(100, 201, 50)
            }

In [20]:
grid = GridSearchCV(LR(), parametrs, cv = 5)
grid.fit(tridf, train_target)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_iter': range(100, 201, 50),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [21]:
grid.best_params_

{'max_iter': 150, 'penalty': 'l1'}

In [27]:
model = LR(max_iter = 100, penalty = 'l1')
model.fit(tridf, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
x = model.predict(test_idf)
f1_score(test_target, x)

0.7591539022611232

##### Нашли лучшие параметры для логистической регрессии. Опробуем модель К ближайших соседей

##### Общая цель - f1 метрика больше 0.75, достигнута, а значит мы создали модель, при которой можем успешно выявлять токсичные комментарии, пускай и не идеально